# Understanding the configuraiton files

The solver, problem and the scene can all be set up from an XML configuration file. The IK example uses the configuration file at `{exotica_examples}/resources/configs/example_ik_trajectory.xml`. The content of this file is:

```XML
<IKConfig>

  <IKSolver Name="IK"/>

  <UnconstrainedEndPoseProblem Name="FigureEightTracking">

    <PlanningScene>
      	<Scene>
            <JointGroup>arm</JointGroup>
            <URDF>{exotica_examples}/resources/robots/lwr_simplified.urdf</URDF>
            <SRDF>{exotica_examples}/resources/robots/lwr_simplified.srdf</SRDF>
            <LoadScene>{exotica_examples}/resources/scenes/example_box.scene</LoadScene>
            <Links>
                <Link Name="TargetOffset" Transform="0.8 -0.1 0.5"/>
                <Link Name="Target" Parent="TargetOffset" />
            </Links>
            <Trajectories>
                <Trajectory Link="Target" File="{exotica_examples}/resources/scenes/figure_eight.traj" />
            </Trajectories>
            <AttachLinks>
                <AttachLink Name="Box" Parent="Target" Local="1"/>
            </AttachLinks>
      	</Scene>
    </PlanningScene>
    
    <Maps>
        <EffFrame Name="Position">
            <EndEffector>
                <Frame Link="lwr_arm_6_link" LinkOffset="0 0 0.2 0.7071 0  0.7071 0" Base="Target"/>
            </EndEffector>
        </EffFrame>
    </Maps>

    <Cost>
      <Task Task="Position" Rho="1e4"/>
    </Cost>

    <StartState>0 0 0 0 0 0 0</StartState>
    <NominalState>0 0 0 0 0 0 0</NominalState>
    <W> 7 6 5 4 3 2 1 </W>
  </UnconstrainedEndPoseProblem>

</IKConfig>
```

## The document root
The root element `<IKConfig></IKConfig>` can have an arbitrary name. It only serves as a container for the nested elements. The nested elements can be a collection of or more `MotionSolver` and `PlanningProblem` definitions. If more than one solver/problem are defined, they can be selected by name at load time. If the names are not specified, the first solver and the fist probelm are used.

## The motion solver
The solver used in this config is the `IKSolver`. The following tag 
```XML
<IKSolver Name="IK"/>
```
only specifies the `Name` attribute. You can use this name to select a particular solver if multiple solvers are declared within one file. The `Name` attribute has to be specified for all top level tags. See the [documentation](https://ipab-slmc.github.io/exotica/overview.html#motion-solvers) for the list available solvers.

## The planning problem
The planning problem used in this example is the `UnconstrainedEndPoseProblem` and it's initilised similarly to the solver using the following tag:
```XML
<UnconstrainedEndPoseProblem Name="FigureEightTracking">
```

This type of problem then specifies:
```XML
<PlanningScene>
<Maps>
<Cost>
<StartState>
<NominalState>
<W>
    
```

The `UnconstrainedEndPoseProblem` defines a single pose cost minimisation problem. This means that the output will be a single pose (the IK solution) and the input is the start state and the cost function. See the [documentation](https://ipab-slmc.github.io/exotica/overview.html#planning-problems) for the list of existing problems.

The start state is a space separated vector:
```XML
<StartState>0 0 0 0 0 0 0</StartState>
```

An optional nominal state is also specified. This is used for redundancy resolution as a cost term minimizing the deviation from the `NominalState` by solvers that support nullspace cost minimisation.

The minimisation also scales/normalizes the state space cost using the provided state weighting:
```XML
<W> 7 6 5 4 3 2 1 </W>
```
This vector assigns a weight to each state variable. Higher values result in higher cost when moving that joint.

#### Cost function
The cost function is composed by adding togeter cost terms for each task. Our example uses only one task:
```XML
<Task Task="Position" Rho="1e4"/>
```
The task name is the name of the task map (see below) that this task is using. `Rho` is the task weight that allows you to scale the task cost. Since this task is used for minimizing end-effector error, the `1e4` scaling ensures sub-milimiter tracking accuracy. If multiple tasks are defined, the `Rho` also defines the relative importance of a task. 

The task map is function of the robot state that is in this cased then used as a cost term. You can define any number of task maps inside the `<Maps>` tag. Our example is using the following task map:
```XML
<EffFrame Name="Position">
    <EndEffector>
        <Frame Link="lwr_arm_6_link" LinkOffset="0 0 0.2 0.7071 0  0.7071 0" Base="Target"/>
    </EndEffector>
</EffFrame>
```
This task map is of type `EffFrame` and it's given the name `Position`. The name is used by the task. The purpose of this task map is to define an error based on difference between two spatial frames, the link frame and the base frame. See the [documentation](https://ipab-slmc.github.io/exotica/overview.html#problem-definition) for a full list of the task maps. The `Frame` tag specifies the transformation of the link frame w.r.t. the base frame. When this transformation is identity, the error of the task is zero. See the [documentation](https://ipab-slmc.github.io/exotica/overview.html#system-model) for full description of the frame conventions. This task map defines the transformation of the `lwr_arm_6_link` with an additional offset w.r.t. the `Target` frame, i.e., the error is zero when the end-effector (link 6) coincides with the target frame. All these frames are defined in the scene.

## The planning scene
```XML
<Scene>
    <JointGroup>arm</JointGroup>
    <URDF>{exotica_examples}/resources/robots/lwr_simplified.urdf</URDF>
    <SRDF>{exotica_examples}/resources/robots/lwr_simplified.srdf</SRDF>
    <LoadScene>{exotica_examples}/resources/scenes/example_box.scene</LoadScene>
    <Links>
        <Link Name="TargetOffset" Transform="0.8 -0.1 0.5"/>
        <Link Name="Target" Parent="TargetOffset" />
    </Links>
    <Trajectories>
        <Trajectory Link="Target" File="{exotica_examples}/resources/scenes/figure_eight.traj" />
    </Trajectories>
    <AttachLinks>
        <AttachLink Name="Box" Parent="Target" Local="1"/>
    </AttachLinks>
</Scene>
```
The planning scene has to specify the `URDF`, `SRDF`, and the `JointGroup`. Joint groups are defined in the SRDF and they specify the joints used for planning as well as the type of the base (fixed, plannr, floating). All the other tags are optional.

```XML
<LoadScene></LoadScene>
```
Loads additional scene objects from a MoveIt scene file. You can load multiple scene files as long as their object names are unique across all of them.
```XML
<Links>
    <Link Name="TargetOffset" Transform="0.8 -0.1 0.5"/>
    <Link Name="Target" Parent="TargetOffset" />
</Links>
```
This tag allows you specify additional links such as targets and intermediate named frames used in the task maps. Each link can specify name, parent link name, and a transform from parent. If the parent is not specified or if its `""`, the root frame is used.
```XML
<Trajectories>
    <Trajectory Link="Target" File="{exotica_examples}/resources/scenes/figure_eight.traj" />
</Trajectories>
```
A trajectory can be used to override the transform of alink w.r.t. its parent link, e.g. above, the `Target` frame transform w.r.t. the `TargetOffset` frame will be calculated from the trajectory transform. When you update the scene at a particular time (e.g., by setting the `start_time` of the problem before calling `solve()`) the frame offset will be calculated based on the trajectory. In this case, the target will move in its parnt's frame.
```XML
<AttachLinks>
    <AttachLink Name="Box" Parent="Target" Local="1"/>
</AttachLinks>
```
This tag allows you to attach (change the parent frame of) and environment object/link. In this case, the `Box` (loaded from the scene file) will move together with the `Target` link.

## Initilisation from XML
When loading the solver from an XML file using `Setup.load_solver()` several initialisation steps take place:
 1. The file gets loaded and parsed as XML using the `XMLLoader` class. Any XML parsing issues will be flagged up.
 1. The output will be a generic `Initializer` class (all parsed properties will be parsed as strings). You can create a generic initializer from a python dictionary as well.
 1. The generic `Initializer` will get implicitly converted into a specilized initializer, e.g. `IKSolverInitializer`. At this point, all datatypes will be correctly parsed and any data format issues will be flagged up.
 1. The `IKSolver` will be initialized using the specialized `IKSolverInitializer`.
 1. The `UnconstrainedEndPoseProblem` will be initialized using the specialized `UnconstrainedEndPoseProblemInitializer`. All contained tatastructures will be initialized recursively.
 1. The problem gets assigned to the solver.
 1. The fully initialized solver gets returned.
 
All of these steps can be run separately but the `Setup.load_solver()` method takes care of this for convenience.
See the [documentation](https://ipab-slmc.github.io/exotica/initialization.html) for more details about initializers and the initialization process.
